# Automated ML

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core import Environment
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

import joblib
from azureml.core.model import Model

from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice

from train import clean_data

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-gridstability-automl'

experiment=Experiment(ws, experiment_name)

In [4]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-187665
aml-quickstarts-187665
southcentralus
d7f39349-a66b-446e-aba6-0053c2cf1c11


In [5]:
run = experiment.start_logging()

In [6]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Cluster already exists. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview

The dataset "Electrical Grid Stability Simulated Data" was obtained from the UCI Machine Learning Repository and can be found [here](https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+#).

The dataset consists of 10000 observations of an electric power grid with four nodes in total, one with electricity generation and three with electricity consumption:

![Diagram](img/0-Diagram.png)

The dataset contains the following features (node 1 refers to the electricty producer, whereas nodes 2 to 4 refers to the electricity consumer):

    - tau: reaction time of each participant (1 to 4).
    - p: nominal power consumed or generated of each node (1 to 4). A negative value indicates a net consumption, whereas a positive value indicates net generation.
    - g: coefficient (gamma) proportional to price elasticity (1 to 4).
    - stab: the maximal real part of the characteristic equation root. A positive value indicates that the system is linearly unstable.
    - stabf: the stability label of the system. This is a categorical feature: stable/unstable.

The aim of this project is to develop a machine learning model that can predict the stability of the system based on the features of the dataset.
Therefore, it is a classification problem in which we want to predict the stabf value.

In [7]:
ds = pd.read_csv("data/gridstability.csv")

In [8]:
x, y = clean_data(ds)

We remove the stab column since we want to predict stabf.

In [9]:
ds_clean= pd.concat([x, y], axis=1)
ds_clean.to_csv("data/ds_clean.csv", index=False)

In [10]:
datastore = ws.get_default_datastore()

In [11]:
datastore.upload(src_dir='data', target_path='data')

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 4 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Target already exists. Skipping upload for data/ds_clean.csv
Target already exists. Skipping upload for data/gridstability.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_b683d853ad4c4bd7b813ad234e148a34

In [12]:
ds = Dataset.Tabular.from_delimited_files(path=[(datastore, "data/ds_clean.csv")])

In [13]:
ds.take(5).to_pandas_dataframe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,1
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,0
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,1
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,1
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,1


## AutoML Configuration

### [AutoML Settings](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings)

    - experiment_timeout_minutes: 20 minutes. This is the maximum number of minutes that the experiment will run for in order to avoid extra costs.
    - max_concurrent_iterations: 5 iterations. This is the maximum number of iterations that will be run in parallel.
    - primary_metric: accuracy. This is the metric that will be used to determine the best model. Accuracy is the proportion of true results among the total number of cases examined and it is used in this case because there is no class imbalance.
    - blocked_models: ['XGBoostClassifier']. This is the list of models that will not be evaluated. In this case, I choose to block the XGBoostClassifier model because to see how this setting works.

### [AutoML Config](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py)

    - compute_target: compute_target defined above. This is the compute target that will be used to run the experiment.
    - task: 'classification'. This is the task that we want to perform according to our problem.
    - training_data: ds. This is the dataset that we want to use to train our model. It includes the features and the labels.
    - label_column_name: "stabf". This is the label column name that we want to use to train our model.
    - path: project_folder. This is the path to the project folder.
    - enable_dnn: True. This is the flag that indicates if we want to consider using deep neural networks.
    - enable_early_stopping: True. This is the flag that indicates if we want to stop the experiment once the results seem not to improve with new iterations.
    - featurization: "auto". This indicates that the featurization will be automatically performed.
    - debug_log: "automl_errors.log". This is the file where the debug information will be logged.

In [14]:
project_folder = 'gridstability-runs'

In [15]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target = compute_target,
                             task = "classification",
                             training_data = ds,
                             label_column_name = "stabf",   
                             path = project_folder,
                             enable_dnn = True,
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             enable_voting_ensemble=False,
                             enable_stack_ensemble=False,
                             **automl_settings
                            )

In [16]:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on my-cluster with default configuration
Running on remote compute: my-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|3                             |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbal

## Run Details

In [17]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

In [21]:
best_run, fit_model = remote_run.get_output()

In [22]:
print(best_run)
print('-------------------------')
print(fit_model)
print('-------------------------')
print(remote_run.get_metrics())

Run(Experiment: capstone-gridstability-automl,
Id: AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e_23,
Type: azureml.scriptrun,
Status: Completed)
-------------------------
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=True, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mounts...
                 LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.99, learning_rate=0.08947473684210526, max_bin=150, max_depth=10, min_child_weight=3, min_data_in_leaf=0.06552068965517242, min_split_gain=0.8421052631578947, n_estimators=600, n_jobs=1, num_leaves=179, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None, reg_alpha=0.42105263157894735, reg_lambda=0.0526315789473

In [23]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e_23,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [24]:
best_run.get_metrics()

{'accuracy': 0.9398003087651295,
 'AUC_weighted': 0.9864385566563051,
 'weighted_accuracy': 0.9458640870532795,
 'norm_macro_recall': 0.8654426163596315,
 'AUC_micro': 0.9874677636915004,
 'f1_score_micro': 0.9398003087651295,
 'average_precision_score_macro': 0.984890429715049,
 'f1_score_macro': 0.934548961848095,
 'precision_score_weighted': 0.9396431602881051,
 'balanced_accuracy': 0.9327213081798158,
 'precision_score_macro': 0.9364995357711546,
 'recall_score_macro': 0.9327213081798158,
 'recall_score_weighted': 0.9398003087651295,
 'average_precision_score_weighted': 0.9869925327140902,
 'f1_score_weighted': 0.9396691804048971,
 'average_precision_score_micro': 0.9879068597265926,
 'AUC_macro': 0.9864385566563051,
 'matthews_correlation': 0.8692119976667073,
 'precision_score_micro': 0.9398003087651295,
 'log_loss': 0.15818521174868846,
 'recall_score_micro': 0.9398003087651295,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e_2

In [25]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': 'a731f594239c01253a85c743f7caa5a6bc69adf8',
 'pipeline_spec': '{"objects": [{"class_name": "RobustScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"quantile_range": [10, 90], "with_centering": true, "with_scaling": true}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "LightGBMClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"boosting_type": "gbdt", "colsample_bytree": 0.99, "learning_rate": 0.08947473684210526, "max_bin": 150, "max_depth": 10, "min_child_weight": 3, "min_data_in_leaf": 0.06552068965517242, "min_split_gain": 0.8421052631578947, "n_estimators": 600, "num_leaves": 179, "reg_alpha": 0.42105263157894735, "reg_lambda": 0.05263157894736842, "subsample": 0.5447368421052632}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "a731f594239c01253a85c743f7caa5a6bc69adf8", "module": "sklearn.pipeline", "class_name": "Pipe

In [26]:
best_run.get_details()

{'runId': 'AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e_23',
 'target': 'my-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-28T12:02:21.931444Z',
 'endTimeUtc': '2022-02-28T12:02:25.597339Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': 'a731f594239c01253a85c743f7caa5a6bc69adf8',
  'pipeline_spec': '{"objects": [{"class_name": "RobustScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {"quantile_range": [10, 90], "with_centering": true, "with_scaling": true}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "LightGBMClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"boosting_type": "gbdt", "colsample_bytree": 0.99, "learning_rate": 0.08947473684210526, "max_bin": 150, "max_depth": 10, "min_child_weight": 3, "min_data_in_leaf": 0.06552068965517242, "min_split_gain": 0.8421052631578947, "n_estimators": 600, "num_leaves": 179, "reg_alpha": 0.421052631

In [27]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.'],
 'matthews_correlation': 0.8692119976667073,
 'AUC_macro': 0.9864385566563051,
 'average_precision_score_micro': 0.9879068597265926,
 'recall_score_weighted': 0.9398003087651295,
 'balanced_accuracy': 0.9327213081798158,
 'precision_score_macro': 0.9364995357711546,
 'precision_score_micro': 0.9398003087651295,
 'average_precision_score_weighted': 0.9869925327140902,
 'recall_score_macro': 0.9327213081798158,
 'average_precision_score_macro': 0.984890429715049,
 'precision_score_weighted': 0.93964

In [28]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'environment.yml')

In [29]:
best_run.download_file('outputs/model.pkl', 'best_automl_model.pkl')

In [30]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-gridstability-automl,AutoML_fba3e28a-e280-42db-bb32-7e2bf0a91e9e_23,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [31]:
joblib.dump(fit_model, filename="outputs/best_model_automl.pkl")

['outputs/best_model_automl.pkl']

## Model Deployment

In [32]:
model_name = best_run.properties['model_name']

In [33]:
print(model_name)

AutoMLfba3e28ae23


In [34]:
model = remote_run.register_model(model_name = model_name)

In [35]:
service_name = 'gridstability-api'
script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)
best_run_env = best_run.get_environment()
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run_env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-28 12:05:12+00:00 Creating Container Registry if not exists.
2022-02-28 12:05:12+00:00 Registering the environment.
2022-02-28 12:05:12+00:00 Use the existing image.
2022-02-28 12:05:12+00:00 Generating deployment configuration.
2022-02-28 12:05:13+00:00 Submitting deployment to compute.
2022-02-28 12:05:15+00:00 Checking the status of deployment gridstability-api..
2022-02-28 12:07:06+00:00 Checking the status of inference endpoint gridstability-api.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [36]:
%run endpoint.py

{"result": [1]}


In [37]:
service.delete()